# **Frameworks**

In [33]:
import numpy as np
import matplotlib.pyplot as plt
import random as rnd

rnd.seed(42)

# **Constants**

## Tourist relevant data

In [ ]:
START_YEAR = 2025
PASSENGERS_PER_YEAR = [214633.4624,208297.2555,209173.5449,213131.6665,206197.0777,205981.4078,212065.4495,216918.4961,217704.3512,208795.952,207959.4197,210968.1772,210601.0115,211754.169,209552.6006,217294.4276,214613.519,208637.98,209413.9192,208329.4301,207396.322,209640.682,210289.1261,220871.3051,204546.983,207716.9742]
JOB_START =  # Amount of jobs xcreated by tourism in the activies sector
VALUE_START =  # Amount of GVA (mio. kr.) created by tourism in the activies sector
REVENUE_START =  # Amount of revenue (mio. kr.) created by tourism in the activies sector

## Boat relevant data

In [35]:
YEARS = np.arange(START_YEAR, START_YEAR + len(PASSENGERS_PER_YEAR))
BOAT_TRIPS_TIME = 1.33 # Hours per trip
MINUTES_WITH_SEALS = 60
DAY_SIGHTSEEING = 365
SEAL_POPULATION_SIZE = 500
IMPACT = 20.0

In [36]:
HIRE_BOAT = {"Power used": 15,       # kW
            "Passenger_per_boat": 7, # passengers without captain
            "SFC": 215e-3,           # kg/kWh
            "CO2": 3190e-3,          # kg/(kg fuel)
            "CH4": 0.05e-3,          # kg/(kg fuel)
            "N2O": 0.08e-3           # kg/(kg fuel)
            }  

COASTAL_BOAT = {"Power used": 17,    # kW
            "Passenger_per_boat": 11,# passengers without captain
            "SFC": 425e-3,           # kg/kWh
            "CO2": 3135e-3,          # kg/(kg fuel)
            "CH4": 1.7e-3,           # kg/(kg fuel)
            "N2O": 0.08e-3           # kg/(kg fuel)
            }  


## Emission data

In [37]:
CO2_EQ_FACTORS = {"CO2": 1, "CH4": 25, "N2O": 298}

## NPS Score

In [ ]:
def calculate_NPS(nps_score):
    detractors = nps_score[0] + nps_score[1] +nps_score[2]+nps_score[3]+nps_score[4]+nps_score[5]+nps_score[6]     # 0–6
    promoters = nps_score[9] + nps_score[10]
    total_responses = detractors + promoters + nps_score[7] + nps_score[8]

    return (promoters-detractors) / total_responses

def calculate_penalty(loss, encounter_prob, cancelled, co2):
    p_enc = min(encounter_prob, 1)
    p_cancel = min(cancelled / 50, 1)
    p_co2 = min(co2 / 5000, 1)

    seal_effect = (loss - 1.0) * 2     

    base_penalty = (
        -0.05 * p_enc +
        0.02 * p_cancel +
        0.01 * p_co2
    )

    adjusted_penalty = base_penalty - (0.13 * seal_effect)

    return max(0, min(1, adjusted_penalty))


def update_nps_distribution(nps, penalty):
    updated = nps.copy()
    
    for score in range(10, 0, -1):
        shift_amount = updated[score] * penalty
        updated[score] -= shift_amount
        updated[score - 1] += shift_amount

    return updated


# **Functions**

In [ ]:
def growth_symmetric(impact_pct, r_max=0.2, k=0.5, threshold=18.4):
    x = k * (impact_pct - threshold)
    S = 1.0 / (1.0 + np.exp(-x))
    return r_max * (1.0 - 2.0 * S)

# Soemthing

In [ ]:
def calculate_weather():
    days_with_rain = rnd.randint(80, 120)
    rain_effect_factor = 1 - (days_with_rain / 365) * rnd.uniform(0.05, 0.15)

    return rain_effect_factor

def calculate_tourist(passengers, boat_passenger_size, nps_list, idx, seal_growth = 0.0):
    NPS_effect = nps_list[idx]/nps_list[0]
    tourism_number = passengers * (0.1 * (1 + seal_growth) * (NPS_effect)) 
    boat_trips = np.round(tourism_number / boat_passenger_size)
    return boat_trips, tourism_number

def calculate_tourist_rain_effect(boat_trips, boat_passenger_size, tourism_number, rain_effect_factor):
    boat_trips_rain     = boat_trips * rain_effect_factor
    cancelled_trips     = boat_trips_rain * (1 - rain_effect_factor) 
    tourism_number_rain = tourism_number - cancelled_trips * boat_passenger_size
    return boat_trips_rain, cancelled_trips, tourism_number_rain

def calculate_boat_stats(boat_trips, boat_power, boat_sfc):
    boat_trips_per_day = boat_trips / DAY_SIGHTSEEING
    boat_trips_hours = boat_trips * BOAT_TRIPS_TIME
    boat_trips_kWh = boat_trips_hours * boat_power
    fuel = boat_trips_kWh * boat_sfc
    return fuel, boat_trips_per_day

In [41]:


def run_simulation(boat):
    BOAT = boat # Define boat
    NPS_SCORE = {0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0, 4: 0.0, 5: 0.0, 6: 1.0, 7: 9.0, 8: 23.0, 9:38.0, 10: 29.0}
    # --- Economic and local statistics
    job_list = [JOB_START*0.1]
    value_list = [VALUE_START*0.1]
    spendage_list = [REVENUE_START*0.1]
    # --- Tourism statistics
    tourism = []
    actual_tourists = []
    boat_trips_year = []
    actual_boat_trips_year = []
    # --- Emission statistics
    co2_list = []
    ch4_list = []
    n2o_list = []
    co2_eq_list = []
    # --- Seal statistics
    seal_population_size = SEAL_POPULATION_SIZE
    seal_population_size_list = [SEAL_POPULATION_SIZE]
    seal_growth = [SEAL_POPULATION_SIZE]
    # --- NPS score list
    nps_list = [calculate_NPS(NPS_SCORE)]

    for i, year in enumerate(YEARS):
        # Calculate trips and tourism numbers
        if i == 0:
            boat_trips, tourism_number  = calculate_tourist(PASSENGERS_PER_YEAR[i], BOAT["Passenger_per_boat"], nps_list, i)
        else:
            growth = seal_growth[i] / seal_growth[0] - 1
            boat_trips, tourism_number  = calculate_tourist(PASSENGERS_PER_YEAR[i], BOAT["Passenger_per_boat"], nps_list, i, seal_growth=growth)

        # Incorporate weather conditions
        rain_effect_factor = calculate_weather()

        # Calculate new boat trips
        boat_trips_rain, cancelled_trips, tourism_number_rain = calculate_tourist_rain_effect(boat_trips, BOAT["Passenger_per_boat"], tourism_number, rain_effect_factor)

        # ---- Update Tourism statistics
        tourism.append(PASSENGERS_PER_YEAR[i]*0.1) # Without any effect
        actual_tourists.append(tourism_number_rain)

        # ---- Update economic and local statistics
        if i > 0:
            # Economic impact on jobs, value and spendage
            tourism_impact = tourism_number_rain/actual_tourists[i-1]
            jobs_change = np.round(job_list[i-1] * (tourism_impact))
            spendage_change = spendage_list[i-1] * tourism_impact * (1 + rnd.uniform(0.015,0.025))
            job_list.append(jobs_change)
            spendage_list.append(spendage_change)

            value_change = value_list[i-1] * spendage_list[i]/spendage_list[i-1] * rnd.uniform(0.95,1.05)
            value_list.append(value_change)

        # ---- Update Boat statistics
        boat_trips_year.append(boat_trips) # without any effet
        actual_boat_trips_year.append(boat_trips_rain)

        # ---- Emission statistics
        fuel, boat_trips_per_day = calculate_boat_stats(boat_trips_rain, BOAT["Power used"], BOAT["SFC"] )

        co2 = fuel * BOAT["CO2"]  
        ch4 = fuel * BOAT["CH4"]
        n2o = fuel * BOAT["N2O"]
        co2_eq = co2 * CO2_EQ_FACTORS["CO2"] + ch4 * CO2_EQ_FACTORS["CH4"] + n2o * CO2_EQ_FACTORS["N2O"]
    
        co2_list.append(co2)
        ch4_list.append(ch4)
        n2o_list.append(n2o)
        co2_eq_list.append(co2_eq)

        # ---- Seal statistics
        feeding_overlap = np.exp(-0.85+0.21*boat_trips_per_day)
        hours_with_seals = DAY_SIGHTSEEING*MINUTES_WITH_SEALS/60
        chance_for_same_encounter = 1 - (1 - 1/seal_population_size) ** (boat_trips_per_day*50)
        loss = 1 / (1 + np.exp(-(-4.97 + 1.76 * feeding_overlap + 0.0036 * hours_with_seals * chance_for_same_encounter)))
        loss = loss*100
        r = growth_symmetric(loss, r_max=0.10, k=0.1, threshold=18.4)
        seal_population_size = seal_population_size * (1 + r)

        seal_growth.append(seal_population_size)
        seal_population_size_list.append(seal_population_size)

        # ---- NPS Score modification:
        penalty = calculate_penalty(
                loss=seal_growth[i+1]/seal_growth[i],
                encounter_prob=chance_for_same_encounter,
                cancelled=cancelled_trips,
                co2=co2_eq
            )
        NPS_SCORE = update_nps_distribution(NPS_SCORE, penalty)
        nps_list.append(calculate_NPS(NPS_SCORE))

    
    return nps_list, seal_population_size_list, co2_list, ch4_list, n2o_list, co2_eq_list, tourism, actual_tourists, boat_trips_year, actual_boat_trips_year, job_list, value_list, spendage_list



        